## Config

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        # "model": ["gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
        "model": ["gpt-4"]
    },
)

llm_config={
    "request_timeout": 600,
    "seed": 42,
    "config_list": config_list,
    "temperature": 0,
    "max_tokens": 8192 * 4,
}

/home/awesomeyuer/miniconda3/envs/autogen-py311-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Construct Agents

In [2]:
urlBaseAddress = "http://192.168.0.103:5000/SamplesData"

ordersDataUrl=f"{urlBaseAddress}/Orders.csv"

customersDataUrl=f"{urlBaseAddress}/Customers.csv"

customersGroupsDataUrl=f"{urlBaseAddress}/CustomersGroups.csv"

productsDataUrl=f"{urlBaseAddress}/Products.csv"

activitiesDataUrl=f"{urlBaseAddress}/Activities.csv"

In [6]:
def is_termination_msg_func(x):
    r = x.get("content", "").rstrip().endswith("TERMINATE")
    print("===============", x)
    # r = True
    return r


user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    # is_termination_msg=lambda x: (x.get("content", "").rstrip().endswith("TERMINATE")),
    is_termination_msg=is_termination_msg_func,
    llm_config=llm_config,
    code_execution_config={"work_dir": "00_awesome_poc"},
    system_message="""
    Reply TERMINATE if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.
    """
)

planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''
    Planner. 
    Suggest a plan. Revise the plan based on feedback from admin and critic, until admin approval.
    The plan may involve an engineer who can write code and others don't write code.
    Explain the plan first. Be clear which step is performed by an engineer.
    ''',
    llm_config=llm_config,
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "paper"},
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message='''Engineer. You follow an approved plan. You write python/shell code to solve tasks. 
    Wrap the code in a code block that specifies the script type. 
    The user can't modify your code. So do not suggest incomplete code which requires others to modify. 
    Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. 
    Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. 
    Suggest the full code instead of partial code or code changes. 
    If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
    analyze the problem, revisit your assumption, 
    collect additional info you need, and think of a different approach to try.
''',
)

ordersAssistant = autogen.AssistantAgent(
    name="OrdersAssistant",
    llm_config=llm_config,
    system_message=f"""
    OrdersAssistant. 
    你负责告知 "Engineer": "Orders" 订单原始数据，从如下地址下载：{ordersDataUrl}，
    以及数据中的字段包括：OrderId,OrderNo,CustomerNo,Product,Price。
    你不用编写代码。
    """
)

productsAssistant = autogen.AssistantAgent(
    name="ProductsAssistant",
    llm_config=llm_config,
    system_message=f"""
    ProductsAssistant. 
    你负责告知 "Engineer": "Orders" 订单原始数据，从如下地址下载：{productsDataUrl}，
    以及数据中的字段包括：Product,Unit,Stock,UnitPrice。
    你不用编写代码。
    """
)

groupchat = autogen.GroupChat(
    agents=[
    user_proxy, 
    planner, 
    engineer,
    executor,
    ordersAssistant, 
    productsAssistant], 
    messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [7]:
user_proxy.initiate_chat(
    manager,
#     message="""
# 获取最受欢迎的商品，然后查询该商品的库存量
# """,

#     message="""
# 获取最受欢迎的商品
# """,

message="""
获取销量最多商品
""",
)

user_proxy (to chat_manager):


获取销量最多商品


--------------------------------------------------------------------------------


InvalidRequestError: This model's maximum context length is 8192 tokens. However, you requested 66149 tokens (613 in the messages, 65536 in the completion). Please reduce the length of the messages or completion.

In [ ]:
# create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    # human_input_mode="TERMINATE",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config,
    code_execution_config={"work_dir": "00_awesome_poc"},
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

In [ ]:
import pandas as pd

csvFilePath = "SamplesData/Orders.csv"

df = pd.read_csv(csvFilePath,sep=',')
df

# df.to_json(orient="records", force_ascii=False)

# `AutoGen` 可能能部分解决 `数据不出境` 问题

## 营销活动策略-落地
1. 读取多个数据表，获得用户、销售、活动洞察
   - 提示词 csv: url
   - 活动? 时间?
1. 总结洞察，形成新活动策略
   - 提示词 ????
1. 拆解策略（活动名，用户群，活动商品，活动折扣等）
   - 提示词 ????
   - 定制、干预 ?????

 

   活动（历史）的：活动表[用户群受众、时间、地点、活动商品列表、活动详情、成本]


1. 执行策略（生成短信/邮件，生成规则，~~发送~~）
   - 仅生成

## 运营活动落地：
1. 每日按照用户输入活动主题，选择用户群
   - 提示词
   - 不管分群!!!!
1. 选择商品，匹配不同用户群
1. 为不同用户群-商品生成活动文案
   - 建议不要文案创作
   - 仅合并merge(活动表[用户群受众、时间、地点、活动商品列表、活动详情])数据生成文案文本?
1. 发送短信
   - 建议仅生成别发送了

## 我们在研究Autogen的时候，需要想想：
- 哪个Agent/角色负责哪一块？
  - assistant
      `Assist Agent` 设计单一职责:
      活动表[用户群受众、时间、地点、活动商品列表、活动详情]
      `Assist` 设计策略考虑的因素：预算、成本、收益、回款时长
- 有几个user_proxy，分别负责什么？
   - `逻辑` 程序退出


In [ ]:
url = "http://192.168.0.103:5000/SamplesData/Orders.csv?{123}"

In [ ]:

user_proxy.initiate_chat(
    assistant,
    # message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",

    # succeed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 然后展示销量最多的商品的名称, 即: "Product"
    # """,

    # succeed
    message=f"""
    请分析订单数据文件: "{url}", 获取最受欢迎的商品
    """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最不受欢迎的商品
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品该促销?
    # """,

    # failed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该停产?
    # """,

    # succeed 订单最少的
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该打折促销?
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?
    # """,

    # message=f"""请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?""",
)

In [ ]:

user_proxy.initiate_chat(
    assistant,
    # message="""Show me the YTD gain of 10 largest technology companies as of today.""",
    # message="""展示今天截至目前为止，10家最大的科技公司的年初至今收益。""",

    # succeed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 然后展示销量最多的商品即: "Product"
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最受欢迎的商品
    # """,

    # succeed
    #  message=f"""
    # 请分析订单数据文件: "{url}", 获取最不受欢迎的商品
    # """,

    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品该促销?
    # """,

    # failed
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该停产?
    # """,

    # succeed 订单最少的
    # message=f"""
    # 请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个商品(Product)该打折促销?
    # """,

    message=f"""
    请分析订单数据文件: "{url}", 请问基于以上数据，能获得哪些洞察，做出什么判断?
    """,

    # message=f"""请分析订单数据文件: "{url}"(没有"quantity"字段) , 哪个昂贵的商品(Product)该打折促销?""",
)